## ASSIGNMENT TWO: JUPYTER NOTEBOOK ##

This file is used in order to generate two csv files. These two files will be used to import the data in to Simio. 

### 2.1 Creating the initialisation file ###

First of all, we import the needed libraries and read in the file provided in assignment two and select a sub dataframe on the preffered road, which is N1.

In [1]:
import pandas as pd
import numpy as np

df_all_bridges = pd.read_excel("BMMS_overview.xlsx",sep='\t',header=0)
df_bridges_n1 = df_all_bridges.loc[df_all_bridges['road'] == "N1"]

Thereafter, we define the clean_bridges function which was made in assignment 1. This function deletes the duplicate or faulty bridges from the existing dataframe.

We also define the columns we would like to use and rename the columns, since they need to overlap with the roads dataframe in order to merge them at a later stadium. 

Every bridge point gets a label "bridge" inserted at the new column called 'what'.

In [2]:
def clean_bridges(panda):
    LRP_not_roads = []
    LRP_previous = ""
    name_previous = ""
    km_previous = 0
    counter_left = 0
    counter_right=0
    
    
    # We loop through every row in the dataframe
    for index, row in panda.iterrows():
        LRP = panda.loc[index, "LRPName"].rstrip().lstrip()
        km = panda.loc[index, "chainage"]
        name = str(panda.loc[index, "name"]).lower()
    
        #we remove the bridge from the dataframe...
        #if the LRP is the same as the LRP before it
        if LRP_previous == LRP:
            panda = panda.drop(index, axis = 0)
            
        #if this bridge is within 5 metres of the previous bridge, it is considered a duplicate
        elif km <= (km_previous + 0.005):
            panda = panda.drop(index, axis = 0) 
            
        #we don't want to compare . to ., because these could be different bridges. Names must match 70%. Bridges must be less
        #than 50m apart
        elif '(r)' in name_previous and abs(km_previous - km) < 0.03:
            if '(l)' in name:
                counter_right += 1
                panda = panda.drop(index, axis = 0) 
        elif '(l)' in name_previous and abs(km_previous - km) < 0.03:
            if '(r)' in name:
                counter_left += 1
                panda = panda.drop(index, axis = 0) 
        
        LRP_previous = LRP
        km_previous = km
        name_previous = name
    
    return(panda)

df_bridges_n1 = clean_bridges(df_bridges_n1)
df_bridges = df_bridges_n1.loc[:,['LRPName','lat','lon','length','condition','structureNr', 'chainage']]
df_bridges.columns = ['lrp','lat','lon','length','condition','structureNr', 'chainage']
df_bridges['what'] = "bridge"

Secondly, we read in the roads file provided in the assignment. We select it on road N1 and subset the columns we would like to use. We also insert a new column and give it the value "roadpoint" to all values (since it are all roadpoints)

In [3]:
df_all_roads = pd.read_csv("_roads3.csv", sep=',')
df_roads_n1 = df_all_roads.loc[df_all_roads['road'] == "N1"]

# subset the cols we want and add a new column that identifies the point
df_roads = df_roads_n1.loc[:,['lrp', 'lat', 'lon','chainage']]
df_roads['what'] = "roadpoint"

Thirdly, since both the dataframes are in the same format, we can append the rows of the road to the bridges dataframe. The only condition it has, that the road is not a duplicate (since it is a bridge, and we don't want duplicate points).

In [4]:
bridge_values = df_bridges['lrp'].to_numpy()

for index, row in df_roads.iterrows():
    if row[0] in bridge_values:
        continue
    
    df_bridges = df_bridges.append(row)

Well, now we have a workable dataframe. However, we need to execute some manual handlings in order to get it in the right format. 

- When we sort the values, LRPS, LRPSa, LRPSb are at the bottom of the dataframe. So we move them up by creating a sub dataframe of the last 3 rows, and append the other ones to it. 
- The starting point and end point should have their own identifier in the column 'what'

In [5]:
df = df_bridges.sort_values(['lrp'])
df_start = df.iloc[-3:]
df.drop(df.tail(3).index,inplace=True)
df = df_start.append(df)
df = df.reset_index(drop = True)

df.loc[(df['lrp'] == "LRPS"),'what'] = "startpoint"
df.loc[(df['lrp'] == "LRPE"),'what'] = "endpoint"



Another thing we would like to add is a column with the processing time given in the assignment.

In [11]:
for index, row in df.iterrows():
    if df.loc[index, 'length'] >= 200:
        df.loc[index, 'Delay'] = 'Random.Triangular(60,120,240)'
    elif df.loc[index, 'length'] >= 50 and df.loc[index, 'length'] < 200:
        df.loc[index, 'Delay'] = 'Random.Uniform(45,90)'
    elif df.loc[index, 'length'] >= 10 and df.loc[index, 'length'] < 50:
        df.loc[index, 'Delay'] = 'Random.Uniform(15,60)'
    elif df.loc[index, 'length'] < 10:
        df.loc[index, 'Delay'] = 'Random.Uniform(10,20)'
    else:
        df.loc[index, 'Delay'] = 0
        
df.to_csv("roads.csv", sep=",",index=False)

### 2.2 Creating the linkage file ###

This file will help to specify the links from sources, nodes, servers and the sink.

We create an empty data frame with the columns object and destination. We fill this dataframe for every LRP point in the previous created dataframe. If the LRP point is a road, the point is a road point and doesn't need to specify 'input' or 'output', since a node has 1 point (unlike a server which has an input and output). 

If the LRP point is not a node, the object needs to link from the output of an object to the input of another object (could be node, or another object).

The for loop creates a 2 dimensional array with the specification of the given LRP and adds it to the empty data frame.

In [6]:
links = pd.DataFrame(columns=['Object','Destination'])

for index, row in df.iterrows():
    new_row = []
    
    if row['what'] == "roadpoint":
        new_row.append(row['lrp'])
    else:
        new_row.append("Ouput@"+row['lrp'])
    
    if index+1 < len(df):
        if df.loc[index+1, 'what'] == "roadpoint":
            new_row.append(df.loc[index+1, 'lrp'])
        else:
            new_row.append("Input@"+df.loc[index+1, 'lrp'])
    
        links.loc[len(links)] = new_row

links.to_csv("destinations.csv", sep=",",index=False)   